# Duke Breast Cancer MRI Dataset Explorer

This notebook helps explore and analyze the Duke Breast Cancer MRI dataset with the following structure:
```
root_dir/
├── Breast_MRI_001/
│   └── patient_directory/
│       ├── dynamic_sequence_1/
│       │   └── *.dcm files
│       ├── dynamic_sequence_2/
│       │   └── *.dcm files
│       └── ...
├── Breast_MRI_002/
└── ...
```

We'll use Python libraries for DICOM file processing and visualization.

## 1. Install Required Libraries

In [ ]:
# Install necessary packages
!pip install pydicom matplotlib numpy pandas seaborn tqdm pillow

## 2. Import Libraries

In [ ]:
import os
import pydicom
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from PIL import Image
from collections import defaultdict, Counter
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')

## 3. Dataset Detection Functions

In [ ]:
def detect_dataset_structure(root_dir):
    """
    Explore the dataset structure and return summary statistics.
    """
    print(f"Analyzing dataset structure in: {root_dir}")
    
    # Dictionary to store structure information
    dataset_info = {
        'total_patients': 0,
        'total_studies': 0,
        'total_sequences': 0,
        'total_dcm_files': 0,
        'patient_ids': [],
        'patient_details': {}
    }
    
    # Check if root directory exists
    if not os.path.exists(root_dir):
        print(f"Error: The directory {root_dir} does not exist.")
        return dataset_info
    
    # Loop through first level directories (Breast_MRI_XXX)
    for patient_folder in sorted(os.listdir(root_dir)):
        patient_path = os.path.join(root_dir, patient_folder)
        
        if not os.path.isdir(patient_path) or not patient_folder.startswith('Breast_MRI_'):
            continue
            
        dataset_info['total_patients'] += 1
        dataset_info['patient_ids'].append(patient_folder)
        
        patient_info = {
            'patient_directory': None,
            'sequences': [],
            'sequence_counts': {},
            'total_files': 0
        }
        
        # Find patient_directory within Breast_MRI_XXX
        patient_subdirs = [d for d in os.listdir(patient_path) if os.path.isdir(os.path.join(patient_path, d))]
        
        if len(patient_subdirs) > 0:
            patient_directory = patient_subdirs[0]  # Assuming there's only one directory per patient
            patient_info['patient_directory'] = patient_directory
            dataset_info['total_studies'] += 1
            
            patient_dir_path = os.path.join(patient_path, patient_directory)
            
            # Loop through sequences
            for sequence_folder in sorted(os.listdir(patient_dir_path)):
                sequence_path = os.path.join(patient_dir_path, sequence_folder)
                
                if os.path.isdir(sequence_path):
                    dataset_info['total_sequences'] += 1
                    patient_info['sequences'].append(sequence_folder)
                    
                    # Count DICOM files
                    dcm_files = [f for f in os.listdir(sequence_path) if f.endswith('.dcm')]
                    file_count = len(dcm_files)
                    patient_info['sequence_counts'][sequence_folder] = file_count
                    patient_info['total_files'] += file_count
                    dataset_info['total_dcm_files'] += file_count
        
        dataset_info['patient_details'][patient_folder] = patient_info
    
    return dataset_info

In [ ]:
def display_dataset_summary(dataset_info):
    """
    Display summary statistics of the dataset.
    """
    print("\n===== DATASET SUMMARY =====")
    print(f"Total patients (Breast_MRI_XXX folders): {dataset_info['total_patients']}")
    print(f"Total studies (patient directories): {dataset_info['total_studies']}")
    print(f"Total sequences: {dataset_info['total_sequences']}")
    print(f"Total DICOM files: {dataset_info['total_dcm_files']}")
    
    if dataset_info['total_patients'] > 0:
        avg_sequences = dataset_info['total_sequences'] / dataset_info['total_patients']
        avg_files = dataset_info['total_dcm_files'] / dataset_info['total_patients']
        print(f"Average sequences per patient: {avg_sequences:.2f}")
        print(f"Average DICOM files per patient: {avg_files:.2f}")
        
        # Collect sequence names across patients
        all_sequences = []
        for patient_id, details in dataset_info['patient_details'].items():
            all_sequences.extend(details['sequences'])
            
        sequence_counts = Counter(all_sequences)
        print("\nMost common sequence names:")
        for seq, count in sequence_counts.most_common(10):
            print(f"  - {seq}: {count} occurrences")
            
        # Sample of patients
        print("\nSample of patient IDs:")
        for patient_id in sorted(dataset_info['patient_ids'])[:5]:
            print(f"  - {patient_id}")
        if len(dataset_info['patient_ids']) > 5:
            print(f"  - ... and {len(dataset_info['patient_ids']) - 5} more")

## 4. Detect the Duke Breast Cancer MRI Dataset

In [ ]:
# Set the root directory path
root_dir = "../data/Duke-Breast-Cancer-MRI"  # Change this to your actual path

# Detect dataset structure
dataset_info = detect_dataset_structure(root_dir)

# Display summary
display_dataset_summary(dataset_info)

## 5. Analyze DICOM Metadata

In [ ]:
def sample_dicom_metadata(dataset_info, root_dir, sample_size=5):
    """
    Sample DICOM files from different patients and extract metadata.
    """
    metadata_samples = []
    sample_count = 0
    
    # Try to get samples from different patients
    for patient_id in dataset_info['patient_ids']:
        if sample_count >= sample_size:
            break
            
        patient_details = dataset_info['patient_details'][patient_id]
        patient_dir = os.path.join(root_dir, patient_id, patient_details['patient_directory'])
        
        # Try each sequence
        for sequence in patient_details['sequences']:
            if sample_count >= sample_size:
                break
                
            sequence_path = os.path.join(patient_dir, sequence)
            dcm_files = [f for f in os.listdir(sequence_path) if f.endswith('.dcm')]
            
            if dcm_files:
                # Get the first DICOM file
                dicom_path = os.path.join(sequence_path, dcm_files[0])
                try:
                    dcm = pydicom.dcmread(dicom_path)
                    metadata_samples.append({
                        'patient_id': patient_id,
                        'sequence': sequence,
                        'file': dcm_files[0],
                        'dicom': dcm
                    })
                    sample_count += 1
                except Exception as e:
                    print(f"Error reading DICOM file {dicom_path}: {e}")
    
    return metadata_samples

In [ ]:
def display_dicom_metadata(metadata_samples):
    """
    Display metadata from sampled DICOM files.
    """
    common_tags = ['PatientName', 'PatientID', 'PatientBirthDate', 'PatientSex',
                   'StudyDescription', 'SeriesDescription', 'Modality',
                   'Manufacturer', 'ManufacturerModelName', 'MagneticFieldStrength',
                   'PixelSpacing', 'SliceThickness', 'RepetitionTime', 'EchoTime']
    
    for i, sample in enumerate(metadata_samples, 1):
        dcm = sample['dicom']
        print(f"\n===== DICOM Sample {i} =====")
        print(f"Patient ID: {sample['patient_id']}")
        print(f"Sequence: {sample['sequence']}")
        print(f"File: {sample['file']}")
        print("\nKey Metadata:")
        
        for tag in common_tags:
            if hasattr(dcm, tag):
                print(f"  - {tag}: {getattr(dcm, tag)}")
        
        print(f"\nImage dimensions: {dcm.Rows} x {dcm.Columns}")
        if hasattr(dcm, 'NumberOfFrames'):
            print(f"Number of frames: {dcm.NumberOfFrames}")
        
        # Add more MRI-specific tags
        mri_tags = ['ScanningSequence', 'SequenceVariant', 'ScanOptions', 'ContrastBolusAgent']
        print("\nMRI-specific tags:")
        for tag in mri_tags:
            if hasattr(dcm, tag):
                print(f"  - {tag}: {getattr(dcm, tag)}")

In [ ]:
# Sample DICOM metadata
metadata_samples = sample_dicom_metadata(dataset_info, root_dir, sample_size=3)

# Display metadata
if metadata_samples:
    display_dicom_metadata(metadata_samples)
else:
    print("No DICOM samples could be extracted. Please check the dataset structure.")

## 6. Visualize MRI Images

In [ ]:
def visualize_dicom_samples(metadata_samples, rows=1):
    """
    Visualize sample DICOM images.
    """
    if not metadata_samples:
        print("No samples to visualize.")
        return
    
    n_samples = min(rows * 2, len(metadata_samples))
    fig, axes = plt.subplots(rows, 2, figsize=(15, 5 * rows))
    if rows == 1:
        axes = [axes]  # Make axes indexable for the single row case
    
    for i, sample in enumerate(metadata_samples[:n_samples]):
        row, col = i // 2, i % 2
        ax = axes[row][col]
        
        dcm = sample['dicom']
        
        # Get the pixel array
        try:
            pixel_array = dcm.pixel_array
            
            # For multi-frame DICOM, display the middle frame
            if len(pixel_array.shape) > 2 and pixel_array.shape[0] > 1:
                middle_frame = pixel_array.shape[0] // 2
                image_data = pixel_array[middle_frame]
                frame_info = f" (frame {middle_frame+1}/{pixel_array.shape[0]})"
            else:
                image_data = pixel_array
                frame_info = ""
            
            # Normalize image for display
            if image_data.max() > 0:
                image_data = (image_data / image_data.max()) * 255
            
            # Display the image
            ax.imshow(image_data, cmap='gray')
            ax.set_title(f"{sample['patient_id']}\n{sample['sequence']}{frame_info}")
            ax.axis('off')
            
        except Exception as e:
            ax.text(0.5, 0.5, f"Error: {str(e)}", ha='center', va='center')
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize sample DICOM images
if metadata_samples:
    visualize_dicom_samples(metadata_samples, rows=2)

## 7. Analyze Sequences and Time Series

In [ ]:
def analyze_dynamic_sequence(patient_id, sequence_name, root_dir, dataset_info):
    """
    Analyze a dynamic sequence of DICOM files.
    """
    patient_details = dataset_info['patient_details'].get(patient_id)
    if not patient_details or sequence_name not in patient_details['sequences']:
        print(f"Sequence {sequence_name} not found for patient {patient_id}")
        return None
    
    sequence_path = os.path.join(root_dir, patient_id, patient_details['patient_directory'], sequence_name)
    dcm_files = sorted([f for f in os.listdir(sequence_path) if f.endswith('.dcm')])
    
    if not dcm_files:
        print(f"No DICOM files found in {sequence_path}")
        return None
    
    print(f"\nAnalyzing sequence {sequence_name} for patient {patient_id}")
    print(f"Total DICOM files: {len(dcm_files)}")
    
    # Load a sample of DICOM files to analyze the sequence
    sample_size = min(10, len(dcm_files))
    sample_indices = np.linspace(0, len(dcm_files)-1, sample_size, dtype=int)
    
    sequence_data = []
    for idx in sample_indices:
        file_path = os.path.join(sequence_path, dcm_files[idx])
        try:
            dcm = pydicom.dcmread(file_path)
            
            # Extract acquisition time if available
            acquisition_time = getattr(dcm, 'AcquisitionTime', f"Unknown-{idx}")
            
            # Check for multi-frame DICOM
            if hasattr(dcm, 'NumberOfFrames') and int(dcm.NumberOfFrames) > 1:
                frames = dcm.pixel_array.shape[0]
                # Get middle frame for analysis
                middle_frame = frames // 2
                image_data = dcm.pixel_array[middle_frame]
                frame_info = f"Multi-frame ({frames} frames)"
            else:
                image_data = dcm.pixel_array
                frame_info = "Single frame"
            
            # Calculate basic image statistics
            sequence_data.append({
                'file': dcm_files[idx],
                'acquisition_time': acquisition_time,
                'frame_info': frame_info,
                'shape': image_data.shape,
                'min': float(np.min(image_data)),
                'max': float(np.max(image_data)),
                'mean': float(np.mean(image_data)),
                'std': float(np.std(image_data))
            })
            
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    return sequence_data

In [ ]:
def visualize_sequence_statistics(sequence_data):
    """
    Visualize statistics of a sequence's DICOM files.
    """
    if not sequence_data:
        print("No sequence data to visualize.")
        return
    
    # Convert to DataFrame for easier visualization
    df = pd.DataFrame(sequence_data)
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot mean intensity across files
    axes[0, 0].plot(df.index, df['mean'], 'o-', color='blue')
    axes[0, 0].set_title('Mean Intensity Across Sequence')
    axes[0, 0].set_xlabel('File Index')
    axes[0, 0].set_ylabel('Mean Intensity')
    axes[0, 0].grid(True)
    
    # Plot min/max intensity range
    axes[0, 1].plot(df.index, df['min'], 'o-', color='blue', label='Min')
    axes[0, 1].plot(df.index, df['max'], 'o-', color='red', label='Max')
    axes[0, 1].fill_between(df.index, df['min'], df['max'], alpha=0.2)
    axes[0, 1].set_title('Min/Max Intensity Range')
    axes[0, 1].set_xlabel('File Index')
    axes[0, 1].set_ylabel('Intensity')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Plot standard deviation
    axes[1, 0].plot(df.index, df['std'], 'o-', color='green')
    axes[1, 0].set_title('Standard Deviation Across Sequence')
    axes[1, 0].set_xlabel('File Index')
    axes[1, 0].set_ylabel('Standard Deviation')
    axes[1, 0].grid(True)
    
    # Display file information
    file_info = '\n'.join([f"{i}: {row['file']} ({row['frame_info']})" 
                          for i, row in df.iterrows()])
    axes[1, 1].text(0.1, 0.5, file_info, fontsize=9, va='center')
    axes[1, 1].set_title('File Information')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Choose a patient and sequence to analyze
# Replace with actual patient ID and sequence name from your dataset
if dataset_info['patient_ids']:
    sample_patient_id = dataset_info['patient_ids'][0]
    
    patient_details = dataset_info['patient_details'][sample_patient_id]
    if patient_details['sequences']:
        sample_sequence = patient_details['sequences'][0]
        
        # Analyze the dynamic sequence
        sequence_data = analyze_dynamic_sequence(sample_patient_id, sample_sequence, root_dir, dataset_info)
        
        # Visualize sequence statistics
        if sequence_data:
            visualize_sequence_statistics(sequence_data)

## 8. Extract and Analyze ROI Data (if available)

In [ ]:
def search_for_segmentations(dataset_info, root_dir):
    """
    Search for segmentation data or ROI information in the dataset.
    """
    print("\nSearching for segmentation data or ROI information...")
    
    segmentation_info = {
        'found': False,
        'locations': [],
        'types': set()
    }
    
    # Common segmentation-related terms to look for in folder/file names
    roi_terms = ['roi', 'segmentation', 'mask', 'contour', 'tumor', 'lesion']
    
    # Check each patient folder for segmentation data
    for patient_id in dataset_info['patient_ids']:
        patient_details = dataset_info['patient_details'][patient_id]
        patient_dir = os.path.join(root_dir, patient_id, patient_details['patient_directory'])
        
        # Check sequence names for ROI-related terms
        for sequence in patient_details['sequences']:
            if any(term in sequence.lower() for term in roi_terms):
                segmentation_info['found'] = True
                segmentation_info['locations'].append({
                    'patient_id': patient_id,
                    'sequence': sequence,
                    'path': os.path.join(patient_dir, sequence)
                })
                segmentation_info['types'].add(sequence)
    
    # Report findings
    if segmentation_info['found']:
        print(f"Found potential segmentation/ROI data in {len(segmentation_info['locations'])} locations.")
        print(f"Types of potential segmentation data: {', '.join(segmentation_info['types'])}")
    else:
        print("No obvious segmentation/ROI data found based on folder naming.")
        print("You may need to check the DICOM metadata for ROI")
        print("No obvious segmentation/ROI data found based on folder naming.")
        print("You may need to check the DICOM metadata for ROI information.")
    
    return segmentation_info

In [ ]:
# Search for segmentation data
segmentation_info = search_for_segmentations(dataset_info, root_dir)

## 9. Analyze DCE-MRI Enhancement Curves (if available)

In [ ]:
def check_for_dce_sequences(dataset_info):
    """
    Identify potential DCE-MRI sequences in the dataset.
    """
    print("\nChecking for potential DCE-MRI sequences...")
    
    dce_terms = ['dce', 'dynamic', 'contrast', 'enhanced', 'pre', 'post']
    potential_dce_sequences = []
    
    for patient_id in dataset_info['patient_ids']:
        patient_details = dataset_info['patient_details'][patient_id]
        
        # Group sequences that might be part of the same DCE-MRI series
        dce_sequences = []
        for sequence in patient_details['sequences']:
            if any(term in sequence.lower() for term in dce_terms):
                dce_sequences.append({
                    'patient_id': patient_id,
                    'sequence': sequence,
                    'file_count': patient_details['sequence_counts'].get(sequence, 0)
                })
        
        if dce_sequences:
            potential_dce_sequences.append({
                'patient_id': patient_id,
                'sequences': dce_sequences,
                'total_sequences': len(dce_sequences)
            })
    
    # Report findings
    if potential_dce_sequences:
        print(f"Found potential DCE-MRI data for {len(potential_dce_sequences)} patients.")
        
        # Display sample
        sample_idx = min(3, len(potential_dce_sequences))
        for i in range(sample_idx):
            patient_data = potential_dce_sequences[i]
            print(f"\nPatient {patient_data['patient_id']} has {patient_data['total_sequences']} potential DCE sequences:")
            for seq in patient_data['sequences']:
                print(f"  - {seq['sequence']} ({seq['file_count']} files)")
    else:
        print("No obvious DCE-MRI sequences found based on folder naming.")
    
    return potential_dce_sequences

In [ ]:
# Check for DCE-MRI sequences
dce_sequences = check_for_dce_sequences(dataset_info)

## 10. Extract Patient Demographics and Clinical Information

In [ ]:
def extract_demographics(dataset_info, root_dir, sample_size=10):
    """
    Extract patient demographics from DICOM metadata.
    """
    print("\nExtracting patient demographics from DICOM metadata...")
    
    demographics = []
    
    # Sample patients
    sample_patients = dataset_info['patient_ids'][:sample_size]
    
    for patient_id in sample_patients:
        patient_details = dataset_info['patient_details'][patient_id]
        patient_dir = os.path.join(root_dir, patient_id, patient_details['patient_directory'])
        
        # Try to find a DICOM file to extract demographics
        for sequence in patient_details['sequences']:
            sequence_path = os.path.join(patient_dir, sequence)
            dcm_files = [f for f in os.listdir(sequence_path) if f.endswith('.dcm')]
            
            if dcm_files:
                # Get the first DICOM file
                dicom_path = os.path.join(sequence_path, dcm_files[0])
                try:
                    dcm = pydicom.dcmread(dicom_path)
                    
                    # Extract demographic information
                    patient_info = {
                        'patient_id': patient_id,
                        'patient_name': str(getattr(dcm, 'PatientName', 'Unknown')),
                        'patient_id_dicom': str(getattr(dcm, 'PatientID', 'Unknown')),
                        'patient_sex': str(getattr(dcm, 'PatientSex', 'Unknown')),
                        'patient_age': str(getattr(dcm, 'PatientAge', 'Unknown')),
                        'study_date': str(getattr(dcm, 'StudyDate', 'Unknown')),
                        'study_description': str(getattr(dcm, 'StudyDescription', 'Unknown'))
                    }
                    
                    demographics.append(patient_info)
                    break  # Found a DICOM file for this patient
                    
                except Exception as e:
                    print(f"Error reading DICOM file {dicom_path}: {e}")
    
    # Convert to DataFrame
    if demographics:
        demographics_df = pd.DataFrame(demographics)
        return demographics_df
    else:
        print("No demographic information could be extracted.")
        return None

In [ ]:
# Extract demographics
demographics_df = extract_demographics(dataset_info, root_dir)

# Display demographics if available
if demographics_df is not None:
    display(demographics_df)

## 11. Data Export Functions

In [ ]:
def export_dicom_to_nifti(patient_id, sequence_name, root_dir, dataset_info, output_dir="nifti_exports"):
    """
    Export DICOM series to NIfTI format.
    Note: This requires additional libraries like dicom2nifti or nibabel.
    """
    try:
        import dicom2nifti
    except ImportError:
        print("Please install dicom2nifti package: pip install dicom2nifti")
        return
    
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    patient_details = dataset_info['patient_details'].get(patient_id)
    if not patient_details or sequence_name not in patient_details['sequences']:
        print(f"Sequence {sequence_name} not found for patient {patient_id}")
        return None
    
    # Construct paths
    sequence_path = os.path.join(root_dir, patient_id, patient_details['patient_directory'], sequence_name)
    output_file = os.path.join(output_dir, f"{patient_id}_{sequence_name}.nii.gz")
    
    try:
        # Convert DICOM to NIfTI
        dicom2nifti.convert_directory(sequence_path, output_file)
        print(f"Successfully exported to {output_file}")
        return output_file
    except Exception as e:
        print(f"Error exporting to NIfTI: {e}")
        return None

In [ ]:
def export_dataset_info_to_csv(dataset_info, output_file="dataset_info.csv"):
    """
    Export dataset information to CSV.
    """
    # Prepare data for CSV
    data = []
    
    for patient_id in dataset_info['patient_ids']:
        patient_details = dataset_info['patient_details'][patient_id]
        
        for sequence in patient_details['sequences']:
            data.append({
                'patient_id': patient_id,
                'patient_directory': patient_details['patient_directory'],
                'sequence': sequence,
                'file_count': patient_details['sequence_counts'].get(sequence, 0)
            })
    
    # Create DataFrame and export
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"Dataset information exported to {output_file}")
    
    return df

In [ ]:
# Export dataset information to CSV
dataset_df = export_dataset_info_to_csv(dataset_info)

# Display summary of exported data
display(dataset_df.head())

## 12. Create 3D Visualizations

In [ ]:
def create_3d_mri_visualization(patient_id, sequence_name, root_dir, dataset_info):
    """
    Create a 3D visualization of an MRI sequence using slices.
    """
    from mpl_toolkits.mplot3d import Axes3D
    
    patient_details = dataset_info['patient_details'].get(patient_id)
    if not patient_details or sequence_name not in patient_details['sequences']:
        print(f"Sequence {sequence_name} not found for patient {patient_id}")
        return None
    
    sequence_path = os.path.join(root_dir, patient_id, patient_details['patient_directory'], sequence_name)
    dcm_files = sorted([f for f in os.listdir(sequence_path) if f.endswith('.dcm')])
    
    if not dcm_files:
        print(f"No DICOM files found in {sequence_path}")
        return None
    
    # Load up to 10 slices for visualization
    max_slices = min(10, len(dcm_files))
    slices = []
    
    print(f"Loading {max_slices} slices for 3D visualization...")
    indices = np.linspace(0, len(dcm_files)-1, max_slices, dtype=int)
    
    for idx in indices:
        file_path = os.path.join(sequence_path, dcm_files[idx])
        try:
            dcm = pydicom.dcmread(file_path)
            pixel_array = dcm.pixel_array
            
            # For multi-frame DICOM, use the middle frame
            if len(pixel_array.shape) > 2 and pixel_array.shape[0] > 1:
                middle_frame = pixel_array.shape[0] // 2
                pixel_array = pixel_array[middle_frame]
            
            # Normalize to 0-1
            if pixel_array.max() > 0:
                pixel_array = pixel_array / pixel_array.max()
            
            slices.append(pixel_array)
    
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    if not slices:
        print("No slices could be loaded for visualization.")
        return None
    
    # Create 3D visualization
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Get dimensions
    z_size = len(slices)
    y_size, x_size = slices[0].shape
    
    # Sample points for visualization (full resolution would be too dense)
    sample_rate = 8  # Sample every 8th pixel
    
    # Create coordinate arrays
    z, y, x = np.meshgrid(np.arange(z_size),
                          np.arange(0, y_size, sample_rate),
                          np.arange(0, x_size, sample_rate),
                          indexing='ij')
    
    # Create value array
    values = np.zeros((z_size, y_size // sample_rate, x_size // sample_rate))
    for i in range(z_size):
        values[i] = slices[i][::sample_rate, ::sample_rate]
    
    # Flatten arrays
    x = x.flatten()
    y = y.flatten()
    z = z.flatten()
    values = values.flatten()
    
    # Filter out low intensity points for clarity
    threshold = 0.2
    mask = values > threshold
    
    # Create scatter plot with color based on intensity
    scatter = ax.scatter(x[mask], y[mask], z[mask], c=values[mask], cmap='viridis', alpha=0.3, s=5)
    
    # Add colorbar
    fig.colorbar(scatter, ax=ax, label='Intensity')
    
    # Set labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z (Slice)')
    ax.set_title(f'3D Visualization - {patient_id}, {sequence_name}')
    
    plt.tight_layout()
    plt.show()
    
    return slices

In [ ]:
# Create 3D visualization for a sample patient and sequence
# Replace with actual patient ID and sequence name from your dataset
if dataset_info['patient_ids']:
    sample_patient_id = dataset_info['patient_ids'][0]
    
    patient_details = dataset_info['patient_details'][sample_patient_id]
    if patient_details['sequences']:
        sample_sequence = patient_details['sequences'][0]
        
        # Create 3D visualization
        create_3d_mri_visualization(sample_patient_id, sample_sequence, root_dir, dataset_info)

## 13. Full Dataset Processing Pipeline

In [ ]:
def process_complete_dataset(root_dir, output_dir="processed_data", max_patients=None):
    """
    Process the complete dataset and organize extracted information.
    """
    print(f"\nProcessing complete dataset at {root_dir}")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Step 1: Detect dataset structure
    dataset_info = detect_dataset_structure(root_dir)
    display_dataset_summary(dataset_info)
    
    # Limit number of patients to process if specified
    if max_patients is not None and max_patients > 0:
        patient_ids = dataset_info['patient_ids'][:max_patients]
    else:
        patient_ids = dataset_info['patient_ids']
    
    print(f"\nProcessing {len(patient_ids)} patients")
    
    # Step 2: Extract demographics
    demographics_df = extract_demographics(dataset_info, root_dir, sample_size=len(patient_ids))
    if demographics_df is not None:
        demographics_df.to_csv(os.path.join(output_dir, "patient_demographics.csv"), index=False)
    
    # Step 3: Process each patient's data
    patients_data = []
    
    for patient_id in tqdm(patient_ids, desc="Processing patients"):
        patient_data = {
            'patient_id': patient_id,
            'sequences': []
        }
        
        patient_details = dataset_info['patient_details'][patient_id]
        patient_dir = os.path.join(root_dir, patient_id, patient_details['patient_directory'])
        
        for sequence in patient_details['sequences']:
            sequence_path = os.path.join(patient_dir, sequence)
            sequence_info = {
                'name': sequence,
                'file_count': patient_details['sequence_counts'].get(sequence, 0)
            }
            
            # Try to read a sample DICOM file for metadata
            dcm_files = [f for f in os.listdir(sequence_path) if f.endswith('.dcm')]
            if dcm_files:
                try:
                    dicom_path = os.path.join(sequence_path, dcm_files[0])
                    dcm = pydicom.dcmread(dicom_path)
                    
                    # Extract key metadata
                    sequence_info['modality'] = getattr(dcm, 'Modality', 'Unknown')
                    sequence_info['series_description'] = getattr(dcm, 'SeriesDescription', 'Unknown')
                    sequence_info['image_size'] = f"{dcm.Rows}x{dcm.Columns}"
                    
                    # Check for multi-frame
                    if hasattr(dcm, 'NumberOfFrames'):
                        sequence_info['frames'] = int(dcm.NumberOfFrames)
                    else:
                        sequence_info['frames'] = 1
                except Exception as e:
                    sequence_info['error'] = str(e)
            
            patient_data['sequences'].append(sequence_info)
            
        patients_data.append(patient_data)
    
    # Step 4: Create summary report
    report_file = os.path.join(output_dir, "dataset_report.txt")
    with open(report_file, 'w') as f:
        f.write(f"Duke Breast Cancer MRI Dataset Report\n")
        f.write(f"Generated on: {pd.Timestamp.now()}\n\n")
        
        f.write(f"Dataset location: {root_dir}\n")
        f.write(f"Total patients: {dataset_info['total_patients']}\n")
        f.write(f"Total studies: {dataset_info['total_studies']}\n")
        f.write(f"Total sequences: {dataset_info['total_sequences']}\n")
        f.write(f"Total DICOM files: {dataset_info['total_dcm_files']}\n\n")
        
        f.write(f"Processed {len(patients_data)} patients\n\n")
        
        # Add sequence type statistics
        sequence_types = []
        for patient in patients_data:
            for seq in patient['sequences']:
                if 'series_description' in seq:
                    sequence_types.append(seq['series_description'])
        
        sequence_counter = Counter(sequence_types)
        f.write("Sequence types in dataset:\n")
        for seq_type, count in sequence_counter.most_common():
            f.write(f"  - {seq_type}: {count} occurrences\n")
    
    print(f"\nProcessing complete. Results saved to {output_dir}")
    print(f"Summary report: {report_file}")
    
    # Export detailed patient data as JSON
    import json
    with open(os.path.join(output_dir, "patients_data.json"), 'w') as f:
        json.dump(patients_data, f, indent=2)
    
    return patients_data

In [ ]:
# Process the complete dataset (or a limited number of patients)
# Set max_patients to limit the number of patients to process (None for all)
# processed_data = process_complete_dataset(root_dir, max_patients=5)

## 14. Conclusion and Next Steps

This notebook has provided a comprehensive exploration of the Duke Breast Cancer MRI dataset structure and contents. From here, you might want to:

1. **Further analyze MRI dynamics**: Implement signal intensity curves analysis for DCE-MRI
2. **Build ML models**: Develop machine learning models for lesion classification or segmentation
3. **Perform advanced image processing**: Implement registration, segmentation, or feature extraction
4. **Standardize the dataset**: Convert all images to a consistent format (NIfTI, etc.)
5. **Create visualizations**: Develop more advanced visualization tools for clinical review

The processed data and extracted metadata provide a solid foundation for further research with this dataset.

## 15. References and Resources

- [PyDICOM Documentation](https://pydicom.github.io/pydicom/stable/)
- [Medical Image Processing with Python](https://www.sciencedirect.com/science/article/pii/S2352914821000137)
- [Breast MRI Analysis Guide](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4950431/)
- [DICOM Standard](https://www.dicomstandard.org/)
- [NiBabel Documentation](https://nipy.org/nibabel/) - For more advanced MRI data processing